#### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
train=pd.read_csv('train.csv')

In [8]:
# Train data analysis and preparation
count=0
wordcountlist=[]
for index, row in train.iterrows():
    count+=1
    words = row['text'].split()
    no_of_words = sum(len(word) for word in words)
    wordcountlist.append(no_of_words)
print(count)

159571


In [9]:
sum1=0
for i in wordcountlist:
    sum1=sum1+i
print(sum1)
avg=sum1/count
print(avg)

51803894
324.6447913467986


In [4]:
train = train.rename(columns={'comment_text': 'text','toxic':'label'})
check_for_nan = train['text'].isnull().values.any()
print (check_for_nan)

False


In [4]:
test=pd.read_csv('test.csv')
test = test.rename(columns={'comment_text': 'text','toxic':'label'})
check_for_nan = train['text'].isnull().values.any()

#### Load the model

In [8]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [10]:
InputExample(guid=None,
             text_a = "Hello, world",
             text_b = None,
             label = 1)

InputExample(guid=None, text_a='Hello, world', text_b=None, label=1)

In [11]:
# Data Preparation
def convert_data_to_examples(train, test, text,label): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[text], 
                                                          text_b = None,
                                                          label = x[label]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[text], 
                                                          text_b = None,
                                                          label = x[label]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'text', 
                                                                           'label')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

text = 'text'
label = 'label'

In [12]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, text, label)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/Users/silaruddin/ENTER/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


#### Hyperparameters

In [14]:
epochs=10
learning_rate=3e-5
epsilon=1e-08

#### Model training

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate, epsilon, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs, validation_data=validation_data)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


    940/Unknown - 16404s 17s/step - loss: 0.1141 - accuracy: 0.9563

In [ ]:
model.save('Bert_for_Toxic')

In [ ]:
import pandas as pd
import numpy as np
from string import digits
modified_data=train.values.tolist()
dataset=[tuple(x) for x in modified_data]

In [ ]:
import numpy as np
import torch
# Adversarial Attack
from textattack.models.wrappers import ModelWrapper

class CustomTensorFlowModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
        #text_array = np.array(text_input_list)
        #preds = self.model(text_array).numpy()
        tf_batch = tokenizer(text_input_list, max_length=128, padding=True, truncation=True, return_tensors='tf')
        tf_outputs = model(tf_batch)
        tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
        tf_predictions=tf_predictions.numpy()
        #logits = tf_predictions
        #logits=np.array(tf_predictions)
        #logits = 1 / (1 + logits)
        #logits = logits.squeeze()
        # Since this model only has a single output (between 0 or 1),
        # we have to add the second dimension.
        #final_preds = torch.stack((tf_predictions, logits))
        return tf_predictions

In [ ]:
CustomTensorFlowModelWrapper(model)('I hate hello')

In [ ]:
pred_sentences=['D aww! He matches this background colour I m seemingly stuck with Thanks']

In [ ]:
model_wrapper = CustomTensorFlowModelWrapper(model)

from textattack.attack_recipes import PWWSRen2019

attack = PWWSRen2019.build(model_wrapper)

results_iterable = attack.attack_dataset(dataset, indices=range(10))
for result in results_iterable:
  print(result.__str__(color_method='ansi'))

In [ ]:
model_wrapper = CustomTensorFlowModelWrapper(model)

from textattack.attack_recipes import BAEGarg2019

attack = BAEGarg2019.build(model_wrapper)

results_iterable = attack.attack_dataset(dataset, indices=range(10))
for result in results_iterable:
  print(result.__str__(color_method='ansi'))

In [ ]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])